In [1]:
import os
import re
import pandas as pd

### Часть 1. Предварительная
Предварительная обработка данных [2 балла]

+ Удалите неоднозначные имена (те имена, которые являются и мужскими, и женскими дновременно), если такие есть;

+ Создайте обучающее и тестовое множество так, чтобы в обучающем множестве классы были сбалансированы, т.е. к классу принадлежало бы одинаковое количество имен;

Читаем наши файлы с именами, убираем из них совпадения

In [61]:
len(female)

5001

In [62]:
male = open('male.txt', 'r')
male = list(map(lambda x:x.strip(), male))

In [63]:
len(male)

2943

In [64]:
commons = set(male)&set(female) #смотрим количество имен, которые встречаются и в мужском, и в женском списке
len(commons)

365

In [65]:
clean_female = list(set(female)^set(commons))

In [66]:
len(clean_female)

4635

In [67]:
clean_male = list(set(male)^set(commons))

In [68]:
len(clean_male)

2578

Создаем обучающее и тестовое множество: т.к. мужских имено меньше, деление типа "тестовая выборка -- 0.3 от всего списка" дало бы неравнозначные результаты. Поэтому делим по индексу.

In [69]:
trainFem = clean_female[:1500]
testFem = clean_female[1500:]

trainMale = clean_male[:1500]
testMale = clean_male[1500:]


In [70]:
trainFemdf = pd.DataFrame(trainFem) #объединяем фреймы с данными, присваиваем им классы
trainFemdf['class'] = 1

testFemdf = pd.DataFrame(testFem)
testFemdf['class'] = 1

trainMaledf = pd.DataFrame(trainMale)
trainMaledf['class'] = 2

testMaledf = pd.DataFrame(testMale)
testMaledf['class'] = 2

trainFemdf.columns = ['name', 'class']
testFemdf.columns = ['name', 'class']
trainMaledf.columns = ['name', 'class']
testMaledf.columns = ['name', 'class']


In [71]:
frames_train = [trainFemdf, trainMaledf]
frames_test = [testFemdf, testMaledf]

train = pd.concat(frames_train)
test = pd.concat(frames_test)

In [72]:
train_df = train[['name']] 
test_df = test[['name']]
y_train = train[['class']] 
y_test = test[['class']]

In [73]:
def lower_text(data):
    clean_line = re.sub('[\W\d_-]+', ' ', data.lower().strip())
    return re.sub(' +', ' ', clean_line)

In [74]:
train_df = train_df.applymap(lower_text) #приводим все к нижнему регистру
test_df = test_df.applymap(lower_text)

### Часть 2. Базовая

Базовый метод классификации [3 балла]

Используйте метод наивного Байеса или логистическую регрессию для классификации имен: в качестве признаков используйте символьные $n$-граммы. Сравните результаты, получаемые при разных $n=2,3,4$ по $F$-мере и аккуратности. В каких случаях метод ошибается?

In [75]:
def get_char(name, b): #функция для разбиения слова на н-граммы: name - это имя, b - число символов
    chargram = [name[i:i+b] for i in range(len(name)-b+1)]
    return chargram

In [76]:
def list_char(names, b): # функция проходится по каждому имени в списке и возвращает список n-грамм
    return [get_char(name, b) for name in names]

In [77]:
traingrams = list_char(train_df['name'], 2)

In [78]:
testgrams = list_char(test_df['name'], 2)

In [79]:
traingrams[:10]

[['gl', 'le', 'en', 'nn', 'ni', 'ie'],
 ['ca', 'at', 'tr', 'ri', 'in', 'na'],
 ['ce', 'es', 'sy', 'ya'],
 ['je', 'ea', 'an', 'nn', 'ni', 'in', 'ne'],
 ['ca', 'ac', 'ci', 'il', 'li', 'ie'],
 ['de', 'ev', 'vo', 'on', 'nn', 'ne'],
 ['na', 'an'],
 ['no', 'or', 'ra', 'ah'],
 ['do', 'or', 'ro'],
 ['je', 'es', 'ss', 'sy']]

In [80]:
X_train = [" ".join(tr) for tr in traingrams] #делаем список, где для каждого имени список н-грамм это строка
X_test = [" ".join(tr) for tr in testgrams]

In [81]:
X_train[:10]

['gl le en nn ni ie',
 'ca at tr ri in na',
 'ce es sy ya',
 'je ea an nn ni in ne',
 'ca ac ci il li ie',
 'de ev vo on nn ne',
 'na an',
 'no or ra ah',
 'do or ro',
 'je es ss sy']

In [82]:
X_train_df = pd.DataFrame(X_train)
X_test_df = pd.DataFrame(X_test)
X_train_df.columns = ['name']
X_test_df.columns = ['name']

In [25]:
#X_train_df

Создаем векторное представление наших имен

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from scipy.spatial.distance import pdist, squareform
import sys
from pprint import pprint

In [83]:
def tfidf_vec(voc=None):
    if(voc):
        vectorizer = TfidfVectorizer()  
        tr = vectorizer.fit_transform(X_train_df["name"]) 
        te = vectorizer.fit_transform(X_test_df["name"]) 
        return (tr, te)
    else:
        vectorizer = TfidfVectorizer() 
        tr = vectorizer.fit_transform(X_train_df["name"]) 
        voc = vectorizer.get_feature_names()
        vectorizer = CountVectorizer(vocabulary=voc) 
        te = vectorizer.fit_transform(X_test_df["name"]) 
        return (tr, te)
train_counts, test_counts = tfidf_vec() 

tfidf_transformer = TfidfTransformer()

x_train = tfidf_transformer.fit_transform(train_counts)
x_test = tfidf_transformer.fit_transform(test_counts)

C:\Users\Ksenia\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Обучаем байесовский классификатор на получившихся данных

In [84]:
from sklearn.naive_bayes import MultinomialNB 
clf = MultinomialNB().fit(x_train, y_train)
print(clf)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


C:\Users\Ksenia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [85]:
predicted = clf.predict(x_test)
len(predicted)

4213

In [29]:
from sklearn.metrics import classification_report
#cмотрим результаты
print(classification_report(y_pred=predicted, y_true=y_test))

             precision    recall  f1-score   support

          1       0.87      0.81      0.84      3135
          2       0.54      0.65      0.59      1078

avg / total       0.79      0.77      0.77      4213



Выглядит неплохо! Попробуем изменить значение n в изначальной функции и посмотрим, что изменится

In [86]:
traingrams = list_char(train_df['name'], 3)
testgrams = list_char(test_df['name'], 3)

In [87]:
X_train = [" ".join(tr) for tr in traingrams]
X_test = [" ".join(tr) for tr in testgrams]

In [88]:
X_train[:10]

['gle len enn nni nie',
 'cat atr tri rin ina',
 'ces esy sya',
 'jea ean ann nni nin ine',
 'cac aci cil ili lie',
 'dev evo von onn nne',
 'nan',
 'nor ora rah',
 'dor oro',
 'jes ess ssy']

In [89]:
X_train_df = pd.DataFrame(X_train)
X_test_df = pd.DataFrame(X_test)
X_train_df.columns = ['name']
X_test_df.columns = ['name']

In [90]:
train_counts, test_counts = tfidf_vec() 

tfidf_transformer = TfidfTransformer()

x_train = tfidf_transformer.fit_transform(train_counts)
x_test = tfidf_transformer.fit_transform(test_counts)

C:\Users\Ksenia\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [91]:
clf = MultinomialNB().fit(x_train, y_train)
predicted = clf.predict(x_test)

print(classification_report(y_pred=predicted, y_true=y_test))

             precision    recall  f1-score   support

          1       0.89      0.82      0.85      3135
          2       0.57      0.71      0.63      1078

avg / total       0.81      0.79      0.80      4213



C:\Users\Ksenia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Так стало немного лучше! Заметим, что женские имена в целом классификатор определяет лучше, чем мужские. А если попробовать 4-граммы?


In [92]:
fourtraingrams = list_char(train_df['name'], 4)
fourtestgrams = list_char(test_df['name'], 4)

In [93]:
X_train = [" ".join(tr) for tr in fourtraingrams]
X_test = [" ".join(tr) for tr in fourtestgrams]

In [94]:
X_train[:10]

['glen lenn enni nnie',
 'catr atri trin rina',
 'cesy esya',
 'jean eann anni nnin nine',
 'caci acil cili ilie',
 'devo evon vonn onne',
 '',
 'nora orah',
 'doro',
 'jess essy']

In [95]:
X_train_df = pd.DataFrame(X_train)
X_test_df = pd.DataFrame(X_test)
X_train_df.columns = ['name']
X_test_df.columns = ['name']

In [96]:
train_counts, test_counts = tfidf_vec() 

tfidf_transformer = TfidfTransformer()

x_train = tfidf_transformer.fit_transform(train_counts)
x_test = tfidf_transformer.fit_transform(test_counts)

C:\Users\Ksenia\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [97]:
clf = MultinomialNB().fit(x_train, y_train)
predicted = clf.predict(x_test)

print(classification_report(y_pred=predicted, y_true=y_test))

             precision    recall  f1-score   support

          1       0.86      0.86      0.86      3135
          2       0.59      0.60      0.60      1078

avg / total       0.79      0.79      0.79      4213



C:\Users\Ksenia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Результативность изменилась, но незначительно. Для женских имен результативность выросла, а для мужских упала. Видимо, n-граммы длиной 3 являются оптимальными.

### Часть 3. Нейросетевая
Используйте реккурентную нейронную сеть с LSTM для решения задачи. В ней может быть несколько слоев с LSTM, несколько слоев c Bidirectional(LSTM). У нейронной сети один выход, определяющий класс имени.

Представление имени для классификации в этом случае: бинарная матрица размера (количество букв в алфавите $\times$ максимальная длина имени). Обозначим его через $x$. Если первая буква имени a, то $x[1][1] = 1$, если вторая – b, то  $x[2][1] = 1$ – то есть, используется one hot encoding. Не забудьте, что все имена должны быть одной длины – maxlen. Это представление имени основано на векторной модели (BoW).

Не забудьте про регуляризацию нейронной сети дропаутами.

Сравните результаты baseline метода, полученные на предыдущем шаге, и результаты нейронной сети по F$-мере и аккуратности. Какой метод лучше и почему?

Сравните результаты, получаемые при разных значениях дропаута, разных числах узлов на слоях нейронной сети по $F$-мере и аккуратности. В каких случаях нейронная сеть ошибается?


In [42]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.recurrent import LSTM
from keras import __version__ as keras_version
from keras.datasets import mnist
from keras.utils import np_utils
from keras import backend as K

import numpy as np


C:\Users\Ksenia\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Приводим очищенные от совпадений списки имен к нижнему регистру

In [101]:
low_male = [m_name.lower() for m_name in clean_male]
low_female = [f_name.lower() for f_name in clean_female]


In [102]:
names_count = len(low_male) + len(low_female)

In [103]:
print('Самое длинное мужское имя:', max(low_male, key=len), len(max(low_male, key=len)), 'символов')

Самое длинное мужское имя: jean-christophe 15 символов


In [104]:
print('Самое длинное женское имя:', max(low_female, key=len), len(max(low_female, key=len)), 'символов')

Самое длинное женское имя: helen-elizabeth 15 символов


In [105]:
maxlen = len(max(low_female, key=len))

Считаем количество используемых символов, присваиваем им номера, создаем словарь

In [106]:
chars = set(  "".join(low_male) + "".join(low_female))
char_index = dict((c, i) for i, c in enumerate(chars))
index_char = dict((i, c) for i, c in enumerate(chars))

In [107]:
index_char

{0: 'f',
 1: 'k',
 2: 'l',
 3: 'e',
 4: 's',
 5: 'x',
 6: 'n',
 7: 'a',
 8: 'o',
 9: 'm',
 10: 't',
 11: '-',
 12: 'p',
 13: 'j',
 14: 'z',
 15: 'w',
 16: 'y',
 17: 'd',
 18: 'q',
 19: 'b',
 20: 'v',
 21: 'c',
 22: "'",
 23: 'g',
 24: 'u',
 25: 'h',
 26: 'r',
 27: ' ',
 28: 'i'}

Создаем бинарную матрицу (пока что пустую) из кол-ва имен, используемых букв и максимальной длины имени

In [108]:
X_train = np.zeros((3000, maxlen, len(chars)), dtype=np.bool)
X_test = np.zeros((names_count-3000, maxlen, len(chars)), dtype=np.bool)
y_train = np.zeros((3000, 2 ), dtype=np.bool)
y_test = np.zeros((names_count-3000, 2 ), dtype=np.bool)

In [109]:
for i, name in enumerate(low_male): #"кодируем" имена, разбиваем на тестовую и обучающую выборки
    for t, char in enumerate(name):
        if i < 1500:
            X_train[i, t, char_index[char]] = 1
        else:
            X_test[i-1500, t, char_index[char]] = 1
    if i < 1500:
        y_train[i, 0 ] = 1
    else:
        y_test[i-1500, 0] = 1

for i, name in enumerate(low_female):
    for t, char in enumerate(name):
        if i < 1500:
            X_train[i+1500, t, char_index[char]] = 1
        else:
            X_test[i-len(low_male)+1500, t, char_index[char]] = 1
    if i < 1500:
        y_train[i+1500, 1 ] = 1
    else:
        y_test[i-len(low_male)+1500, 1] = 1

In [110]:
batch_size = 32

In [53]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report, confusion_matrix
from keras import metrics

In [54]:
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, validation_split=0.1)



Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 2700 samples, validate on 300 samples
Epoch 1/10
2700/2700 [==============================] - 121s 45ms/step - loss: 0.6735 - acc: 0.6544 - val_loss: 0.7030 - val_acc: 0.4900
Epoch 2/10
2700/2700 [==============================] - 117s 43ms/step - loss: 0.5681 - acc: 0.7244 - val_loss: 0.6463 - val_acc: 0.6133
Epoch 3/10
2700/2700 [==============================] - 116s 43ms/step - loss: 0.5299 - acc: 0.7411 - val_loss: 0.5286 - val_acc: 0.7200
Epoch 4/10
2700/2700 [==============================] - 118s 44ms/step - loss: 0.5112 - acc: 0.7504 - val_loss: 0.9617 - val_acc: 0.4367
Epoch 5/10
2700/2700 [==============================] - 119s 44ms/step - loss: 0.5006 - acc: 0.7567 - val_loss: 0.6741 - val_acc: 0.6500
Epoch 6/10
2700/2700 [==============================] - 119s 44ms/step - loss: 0.4902 - acc: 0.7637 - val_loss: 0.5683 - v

In [55]:
score = model.evaluate(X_test, y_test) #проверяем модель на тестовой выборке

4213/4213 [==============================] - 59s 14ms/step


In [56]:
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.7267938192457194
Test accuracy: 0.6582008070258722


In [57]:
preds = model.predict(X_test)

In [111]:
y_classes = preds.argmax(axis=-1)
y_test_ar = y_test.argmax(axis=-1)

In [113]:
print(classification_report(y_pred=y_classes, y_true=y_test_ar))

             precision    recall  f1-score   support

          0       0.64      0.74      0.68      1734
          1       0.79      0.71      0.75      2479

avg / total       0.73      0.72      0.72      4213



Результаты получились несколько хуже, чем при обучении при помощи байесовского метода (там f-мера была 80). Попробуем изменить значение дропаута.

In [114]:
model2 = Sequential()
model2.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model2.add(Dropout(0.4))
model2.add(LSTM(512, return_sequences=False))
model2.add(Dropout(0.4))
model2.add(Dense(2))
model2.add(Activation('softmax'))

model2.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model2.fit(X_train, y_train, batch_size=batch_size, validation_split=0.1)

Train on 2700 samples, validate on 300 samples
Epoch 1/10
2700/2700 [==============================] - 121s 45ms/step - loss: 0.6152 - acc: 0.6463 - val_loss: 0.4992 - val_acc: 0.7467
Epoch 2/10
2700/2700 [==============================] - 116s 43ms/step - loss: 0.5544 - acc: 0.7248 - val_loss: 0.9278 - val_acc: 0.4700
Epoch 3/10
2700/2700 [==============================] - 116s 43ms/step - loss: 0.5217 - acc: 0.7530 - val_loss: 0.5258 - val_acc: 0.7100
Epoch 4/10
2700/2700 [==============================] - 118s 44ms/step - loss: 0.5112 - acc: 0.7530 - val_loss: 0.5949 - val_acc: 0.6967
Epoch 5/10
2700/2700 [==============================] - 119s 44ms/step - loss: 0.4974 - acc: 0.7663 - val_loss: 0.5301 - val_acc: 0.7533
Epoch 6/10
2700/2700 [==============================] - 118s 44ms/step - loss: 0.4791 - acc: 0.7804 - val_loss: 0.7420 - val_acc: 0.5567
Epoch 7/10
2700/2700 [==============================] - 119s 44ms/step - loss: 0.4656 - acc: 0.7756 - val_loss: 0.5218 - val_acc: 0

In [115]:
score2 = model2.evaluate(X_test, y_test)

4213/4213 [==============================] - 54s 13ms/step


In [116]:
print('Test score:', score2[0])
print('Test accuracy:', score2[1])

Test score: 0.6998385692078514
Test accuracy: 0.6280560170899596


In [117]:
preds2 = model2.predict(X_test)

In [118]:
y_classes = preds2.argmax(axis=-1)

In [119]:
print(classification_report(y_pred=y_classes, y_true=y_test_ar))

             precision    recall  f1-score   support

          0       0.62      0.81      0.70      1734
          1       0.83      0.65      0.73      2479

avg / total       0.74      0.71      0.72      4213



Результаты незначительно изменились. Скорректируем количество узлов LSTM на модели со старами параметрами дропаута:

In [120]:
model3 = Sequential()
model3.add(LSTM(64, return_sequences=True, input_shape=(maxlen, len(chars))))
model3.add(Dropout(0.2))
model3.add(LSTM(64, return_sequences=False))
model3.add(Dropout(0.2))
model3.add(Dense(2))
model3.add(Activation('softmax'))

model3.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model3.fit(X_train, y_train, batch_size=batch_size, validation_split=0.1)

Train on 2700 samples, validate on 300 samples
Epoch 1/10
2700/2700 [==============================] - 9s 3ms/step - loss: 0.6047 - acc: 0.6585 - val_loss: 0.8864 - val_acc: 0.4467
Epoch 2/10
2700/2700 [==============================] - 5s 2ms/step - loss: 0.5214 - acc: 0.7441 - val_loss: 0.9465 - val_acc: 0.3833
Epoch 3/10
2700/2700 [==============================] - 5s 2ms/step - loss: 0.5027 - acc: 0.7552 - val_loss: 0.5799 - val_acc: 0.6867
Epoch 4/10
2700/2700 [==============================] - 5s 2ms/step - loss: 0.4941 - acc: 0.7619 - val_loss: 0.6790 - val_acc: 0.6233
Epoch 5/10
2700/2700 [==============================] - 5s 2ms/step - loss: 0.4790 - acc: 0.7778 - val_loss: 0.5563 - val_acc: 0.7200
Epoch 6/10
2700/2700 [==============================] - 5s 2ms/step - loss: 0.4719 - acc: 0.7781 - val_loss: 0.7241 - val_acc: 0.5667
Epoch 7/10
2700/2700 [==============================] - 5s 2ms/step - loss: 0.4715 - acc: 0.7807 - val_loss: 0.7523 - val_acc: 0.5367
Epoch 8/10
2700

In [121]:
score3 = model3.evaluate(X_test, y_test)

4213/4213 [==============================] - 2s 436us/step


In [122]:
preds3 = model3.predict(X_test)

In [123]:
y_classes = preds3.argmax(axis=-1)

In [125]:
print(classification_report(y_pred=y_classes, y_true=y_test_ar))

             precision    recall  f1-score   support

          0       0.63      0.73      0.68      1734
          1       0.79      0.70      0.74      2479

avg / total       0.72      0.71      0.72      4213



In [126]:
print('Test score:', score3[0])
print('Test accuracy:', score3[1])

Test score: 0.6827772585960287
Test accuracy: 0.6536909565630192


In [127]:
model4 = Sequential()
model4.add(LSTM(1024, return_sequences=True, input_shape=(maxlen, len(chars)))) #меняем число нейронов
model4.add(Dropout(0.2))
model4.add(LSTM(1024, return_sequences=False))
model4.add(Dropout(0.2))
model4.add(Dense(2))
model4.add(Activation('softmax'))

model4.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model4.fit(X_train, y_train, batch_size=batch_size, validation_split=0.1)

Train on 2700 samples, validate on 300 samples
Epoch 1/10
2700/2700 [==============================] - 428s 159ms/step - loss: 1.3255 - acc: 0.5022 - val_loss: 1.6390 - val_acc: 0.0000e+00
Epoch 2/10
2700/2700 [==============================] - 426s 158ms/step - loss: 0.7775 - acc: 0.5063 - val_loss: 0.7950 - val_acc: 0.0567
Epoch 3/10
2700/2700 [==============================] - 428s 159ms/step - loss: 0.7383 - acc: 0.5289 - val_loss: 1.5578 - val_acc: 0.0000e+00
Epoch 4/10
2700/2700 [==============================] - 431s 160ms/step - loss: 0.7879 - acc: 0.5048 - val_loss: 1.3579 - val_acc: 0.0000e+00
Epoch 5/10
2700/2700 [==============================] - 434s 161ms/step - loss: 0.8504 - acc: 0.5019 - val_loss: 0.5839 - val_acc: 1.0000
Epoch 6/10
2700/2700 [==============================] - 433s 160ms/step - loss: 0.7851 - acc: 0.5096 - val_loss: 0.2932 - val_acc: 1.0000
Epoch 7/10
2700/2700 [==============================] - 430s 159ms/step - loss: 0.7494 - acc: 0.5474 - val_loss: 

In [128]:
score4 = model4.evaluate(X_test, y_test)

4213/4213 [==============================] - 189s 45ms/step


In [129]:
print('Test score:', score4[0])
print('Test accuracy:', score4[1])

Test score: 0.9711225434886626
Test accuracy: 0.5988606693567529


In [130]:
preds4 = model4.predict(X_test)
y_classes = preds4.argmax(axis=-1)

In [131]:
print(classification_report(y_pred=y_classes, y_true=y_test_ar))

             precision    recall  f1-score   support

          0       0.60      0.73      0.66      1734
          1       0.77      0.66      0.71      2479

avg / total       0.70      0.69      0.69      4213



Опять почти ничего не изменилось. В целом байесовскй классификатор показал более точные результаты.